In [ ]:
import pandas as pd
import requests

In [ ]:
population_ger=pd.read_csv("data/12411-02-03-5-B.csv",sep=";",encoding="ISO-8859-1",skiprows=6)

In [ ]:
# first 8 and last 4 rows contain meta data -> drop
population_ger.drop(population_ger.tail(4).index,inplace=True)
# information not needed on larger age range 
population_ger.drop(index=0,inplace=True)
# same columns don't have a proper name
population_ger.rename(columns={"Unnamed: 0":"county_id","Unnamed: 1": "county_name"},inplace=True)

In [ ]:
population_ger=population_ger[~(population_ger.loc[:,population_ger.columns[2:]]=="-").all(axis=1)]

In [ ]:
splitted=population_ger.county_name.str.split(",",expand=True)

In [ ]:
splitted.loc[splitted[2].notna(),1]=splitted.loc[splitted[2].notna(),2]

In [ ]:
population_ger["county_name"]=splitted[0]
population_ger["county_type"]=splitted[1]

In [ ]:
gender_id_to_sheet_map={"a":"Insgesamt","m":"männlich","w":"weiblich"}

In [ ]:
id_vars=["county_id","county_name","county_type"]

In [ ]:
df_l=[]
for k,v in gender_id_to_sheet_map.items():
    gender_columns=id_vars +population_ger.columns[population_ger.columns.str.contains(v)].tolist()
    gender_df=population_ger[gender_columns].reset_index(drop=True)
    gender_df.columns=id_vars+gender_df.iloc[0,~(gender_df.columns.isin(id_vars))].tolist()
    gender_df.drop(0,inplace=True)
    gender_df=gender_df.melt(id_vars=id_vars,var_name="range")
    gender_df["age_from"]=pd.to_numeric(gender_df.range.str.extract(r'^(\d+)')[0])
    gender_df["age_to"]=pd.to_numeric(gender_df.range.str.extract(r'unter (\d+)')[0])
    gender_df.loc[gender_df.age_to==3,"age_from"]=0
    gender_df["gender"]=k
    df_l.append(gender_df)
population_processed=pd.concat(df_l)

In [ ]:
address="Schwarzwald-Baar-Kreis,+Germany"
api_key=""
#r=requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}")

In [ ]:
for i,lk in enumerate(population_processed.county_name.unique()):
    if r_l[i].status_code!=200:
        print(lk)
    with open(f"data/geolocation/{lk}.json","w") as f:
        json.dump(r_l[i].json(),f)

In [ ]:
name_geo=[]
for i,lk in enumerate(population_processed.county_name.unique()):
    d=r_l[i].json()["results"][0]["geometry"]["location"]
    d["county_name"]=lk
    name_geo.append(d)


In [ ]:
geolocation=pd.DataFrame(name_geo)

In [ ]:
population_processed=population_processed.merge(geolocation,on=["county_name"])

In [ ]:
population_processed.to_csv("data/processed/population_germany.csv",index=False)